In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/customer-analytics/Train.csv')

### ID is unique column and can be dropped

In [ ]:
df.drop('ID',axis=1,inplace=True)

In [ ]:
df.info()

### There are no missing values

In [ ]:
df.describe()

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot(data=df,x="Discount_offered")

## Checking the correlation

In [ ]:
sns.heatmap(df.corr(),annot=True)

#### from above observation weight in gms and discounted offered higly correlated with ontime delivery

### Create dummies for categorical variables

In [ ]:
df['Warehouse_block'].value_counts()

In [ ]:
df['Mode_of_Shipment'].value_counts()

In [ ]:
df['Product_importance'].value_counts()

In [ ]:
df['Gender'].value_counts()

In [ ]:
dummy_var=pd.get_dummies(df[['Warehouse_block','Mode_of_Shipment','Product_importance','Gender']])

In [ ]:
dummy_var

In [ ]:
df.drop(['Warehouse_block','Mode_of_Shipment','Product_importance','Gender'],axis=1,inplace=True)

In [ ]:
df=pd.concat([df,dummy_var],axis=1)

In [ ]:
df.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (30,30))
sns.heatmap(df.corr(),annot=True)
plt.show()

### product_importance_medium and product_importance_low are highly correlated

#### 

In [ ]:
df.drop('Product_importance_low',axis=1,inplace=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import re

In [ ]:
rf=RandomForestClassifier(random_state=42)

In [ ]:
Y=df.pop('Reached.on.Time_Y.N')
X=df

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

### Gridsearch CV to find the best params

In [ ]:
param_grid = { 
    'n_estimators': [10,20,30,40,50,100],
    'max_depth' : [2,3,4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [ ]:
CV_rfc = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, Y_train)

In [ ]:
CV_rfc.best_params_

In [ ]:
rfb=RandomForestClassifier(random_state=42,max_depth=5,n_estimators=20,criterion='entropy')

In [ ]:
rfb.fit(X_train,Y_train)

In [ ]:
Y_pred=rfb.predict(X_test)

In [ ]:
print("Accuracy for Random Forest on CV data: ",accuracy_score(Y_test,Y_pred))